<a href="https://colab.research.google.com/github/RV05/Classification-of-disasters-using-modified-architrctures/blob/main/Copy_of_disaster_Resnet_Adam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd "/content/drive/MyDrive/SCAAI_Rohit_volety_Problem_Statement_3/deep learning"

/content/drive/MyDrive/deep learning


In [ ]:
import matplotlib
matplotlib.use("Agg")
# import the necessary packages
import config
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import Adagrad
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.applications import EfficientNetB7
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import VGG19


from tensorflow.keras.applications import ResNet101
from tensorflow.keras.layers import GlobalAveragePooling2D

from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
# ap = argparse.ArgumentParser()
# ap.add_argument("-p", "--plot", type=str, default="plot.png",
# 	help="path to output loss/accuracy plot")
# args = vars(ap.parse_args())
# # determine the total number of image paths in training, validation,
# # and testing directories


In [ ]:
totalTrain = len(list(paths.list_images(config.TRAIN_PATH)))
totalVal = len(list(paths.list_images(config.VAL_PATH)))
totalTest = len(list(paths.list_images(config.TEST_PATH)))
trainAug = ImageDataGenerator(
	# rotation_range=25,
	# zoom_range=0.1,
	# width_shift_range=0.1,
	# height_shift_range=0.1,
	# shear_range=0.2,
	# horizontal_flip=True,
  # vertical_flip=True,
	# fill_mode="nearest"
  )

In [ ]:
# initialize the validation/testing data augmentation object (which
# we'll be adding mean subtraction to)
valAug = ImageDataGenerator()

In [ ]:
# define the ImageNet mean subtraction (in RGB order) and set the
# the mean subtraction value for each of the data augmentation
# objects
# mean = np.array([123.68, 116.779, 103.939], dtype="float32")
# trainAug.mean = mean
# valAug.mean = mean

In [ ]:
trainGen = trainAug.flow_from_directory(
	config.TRAIN_PATH,
	class_mode="categorical",
	target_size=(200, 200),
	color_mode="rgb",
	shuffle=True,
	batch_size=config.BS)
# initialize the validation generator
valGen = valAug.flow_from_directory(
	config.VAL_PATH,
	class_mode="categorical",
	target_size=(200, 200),
	color_mode="rgb",
	shuffle=False,
	batch_size=config.BS)
# initialize the testing generator
testGen = valAug.flow_from_directory(
	config.TEST_PATH,
	class_mode="categorical",
	target_size=(200, 200),
	color_mode="rgb",
	shuffle=False,
	batch_size=config.BS)

Found 3309 images belonging to 6 classes.
Found 31 images belonging to 6 classes.
Found 1022 images belonging to 6 classes.


In [ ]:
print("[INFO] preparing model...")
baseModel = ResNet101(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(200, 200, 3)))
# construct the head of the model that will be placed on top of the
# the base model
headModel = baseModel.output
headModel = AveragePooling2D()(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(256, activation="relu")(headModel)
headModel = Dense(256, activation="relu")(headModel)
headModel = Dense(256, activation="relu")(headModel)
headModel = Dense(256, activation="relu")(headModel)
headModel = Dense(256, activation="relu")(headModel)

headModel = Dropout(0.5)(headModel)
headModel = Dense(len(config.CLASSES), activation="softmax")(headModel)
# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)
# loop over all layers in the base model and freeze them so they will
# *not* be updated during the training process
for layer in baseModel.layers:
	layer.trainable = False
# compile the model
opt = Adam(lr=config.INIT_LR, decay=config.INIT_LR / config.NUM_EPOCHS)
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])
# train the model

[INFO] preparing model...
171458560/171446536 [==============================] - 2s 0us/step


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [ ]:

# load the ResNet-50 network, ensuring the head FC layer sets are left
# off

print("[INFO] training model...")
H = model.fit_generator(
	trainGen,
	steps_per_epoch=totalTrain // config.BS,
	validation_data=valGen,
	validation_steps=totalVal // config.BS,
	epochs=config.NUM_EPOCHS)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


[INFO] training model...
Epoch 1/200
413/413 [==============================] - 1007s 2s/step - loss: 0.8976 - accuracy: 0.6731 - val_loss: 1.1096 - val_accuracy: 0.7083
Epoch 2/200
413/413 [==============================] - 28s 67ms/step - loss: 0.3775 - accuracy: 0.8785 - val_loss: 1.2878 - val_accuracy: 0.7083
Epoch 3/200
413/413 [==============================] - 27s 66ms/step - loss: 0.1887 - accuracy: 0.9373 - val_loss: 1.3568 - val_accuracy: 0.7083
Epoch 4/200
413/413 [==============================] - 27s 65ms/step - loss: 0.0959 - accuracy: 0.9703 - val_loss: 1.7730 - val_accuracy: 0.7083
Epoch 5/200
413/413 [==============================] - 27s 65ms/step - loss: 0.0673 - accuracy: 0.9758 - val_loss: 1.8799 - val_accuracy: 0.6667
Epoch 6/200
413/413 [==============================] - 27s 65ms/step - loss: 0.0881 - accuracy: 0.9727 - val_loss: 2.4740 - val_accuracy: 0.7083
Epoch 7/200
413/413 [==============================] - 27s 66ms/step - loss: 0.0578 - accuracy: 0.9794 - 

In [ ]:
print("[INFO] evaluating network...")
testGen.reset()
predIdxs = model.predict_generator(testGen,
	steps=(totalTest // config.BS) + 1)
# for each image in the testing set we need to find the index of the
# label with corresponding largest predicted probability
predIdxs = np.argmax(predIdxs, axis=1)
# show a nicely formatted classification report
print(classification_report(testGen.classes, predIdxs,
	target_names=testGen.class_indices.keys()))
# serialize the model to disk
print("[INFO] saving model...")
model.save(config.MODEL_PATH, save_format="h5")
N = config.NUM_EPOCHS

[INFO] evaluating network...


/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:2035: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


              precision    recall  f1-score   support

        fire       0.89      0.79      0.83       200
       human       0.84      0.76      0.80        42
       infra       0.43      0.87      0.57       210
        land       0.76      0.48      0.59       148
    nodamage       0.25      0.04      0.07       211
       water       0.68      0.82      0.74       211

    accuracy                           0.61      1022
   macro avg       0.64      0.63      0.60      1022
weighted avg       0.60      0.61      0.57      1022

[INFO] saving model...


/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


In [ ]:
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("Resnet_adam_plot.png")

In [ ]:
H.history

{'accuracy': [0.6731293797492981,
  0.8785216808319092,
  0.9372917413711548,
  0.9703119993209839,
  0.9757649302482605,
  0.9727355241775513,
  0.9794001579284668,
  0.9860648512840271,
  0.9827325344085693,
  0.9781884551048279,
  0.9818236827850342,
  0.9797031283378601,
  0.9900030493736267,
  0.9927294850349426,
  0.9912148118019104,
  0.9912148118019104,
  0.9869736433029175,
  0.9781884551048279,
  0.988791286945343,
  0.9881854057312012,
  0.9890941977500916,
  0.9881854057312012,
  0.9912148118019104,
  0.986670732498169,
  0.9897000789642334,
  0.9875795245170593,
  0.9906089305877686,
  0.9930323958396912,
  0.9933353662490845,
  0.9924265146255493,
  0.9921236038208008,
  0.980309009552002,
  0.9897000789642334,
  0.9918206334114075,
  0.9915177226066589,
  0.9945471286773682,
  0.993638277053833,
  0.993638277053833,
  0.9942441582679749,
  0.9933353662490845,
  0.9954559206962585,
  0.9842472076416016,
  0.9815207719802856,
  0.9869736433029175,
  0.9933353662490845,
  0

In [ ]:
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("Resnet_adam_plot.png")